### 1. Loading required python modules and setting path to SHyFT installation

In [1]:
# python modules
import sys
import os

In [2]:
# setting up the environment
sys.path.insert(0,'/geo/nunatak/hycamp/software/shyft_workspace/shyft')

In [3]:
# shyft modules
from shyft.repository.default_state_repository import DefaultStateRepository
from shyft.orchestration.configuration.yaml_configs import YAMLCalibConfig
from shyft.orchestration.simulators.config_simulator import ConfigCalibrator

###2. Configuration of a SHyFT calibration

In [8]:
# set up configuration using *.yaml configuration files
config_file_path = os.path.abspath("./nea-config/neanidelva_calibration.yaml") # here is the *.yaml file
cfg = YAMLCalibConfig(config_file_path, "neanidelva")

In [9]:
# initialize a calibrator-object
calib = ConfigCalibrator(cfg)
n_cells = calib.region_model.size()
state_repos = DefaultStateRepository(calib.region_model.__class__, n_cells)
calib.init()

###3. Running a SHyFT calibration

In [10]:
results = calib.calibrate(cfg.sim_config.time_axis, state_repos.get_state(0), 
                          cfg.optimization_method['name'],
                          cfg.optimization_method['params'], p_vec=None)

KeyError: 'kirchner.c1'

###4. Calibration results
First the Nash-Suttcliffe-efficiency of the calibrated simulation is computed to see the quality of the calibration.
Then the calibrated model parameters are accessed. If they reach the intevall boarders, a warning is printed out.

In [ ]:
# Get NSE of calibrated run:
result_params = []
for i in range(results.size()):
    result_params.append(results.get(i))
print("NSE =", 1-calib.optimizer.calculate_goal_function(result_params))

In [ ]:
# Check out parameters. Print a warning if parameter interval is reached.
diff = 1.0E-3
print("{0:30s} {1:10s} {2:10s} {3:10s} {4:s}".format("PARAM-NAME", "CALIB-PARAM", "MIN", "MAX",
                                                       "INTERVAL-CHECK"))
for i in range(results.size()): 
    p_cal = results.get(i) # calibrated parameter
    p_min = calib.p_min[i] # min parameter
    p_max = calib.p_max[i] # max parameter
    check_interval = (((p_cal-p_min) < diff) or ((p_max-p_cal) < diff)) and not ((p_max - p_min) < diff)
    print("{0:30s} {1:10f} {2:10f} {3:10f} {4}".format(results.get_name(i),p_cal, p_min, p_max,
                                                       "check interval" if check_interval else "ok"))